# Wind Stress

Mean zonal wind stress profile

**Requirements:** The `conda/analysis3-19.10` (or later) module on the VDI (or your own up-to-date cookbook installation).

## Theory

Wind stress, $\tau$, transfers momentum from the atmosphere to the ocean.

## Calculation

In [ ]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
from dask.distributed import Client

In [ ]:
client = Client(n_workers=4)
client

In [ ]:
session = cc.database.create_session()
start_time='1950-01-01'

The following code block shows the zonal- and time-averaged wind stress forcing for each experiment.

In [ ]:
def calc_mean_tau_x(expt):
    
    tau_x = cc.querying.getvar(expt, 'tau_x', session, start_time=start_time)

    mean_tau_x = tau_x.mean('xu_ocean').mean('time')
    mean_tau_x = mean_tau_x.compute()
    mean_tau_x.name = expt
    
    return mean_tau_x

In [ ]:
expts = ['01deg_jra55v13_ryf9091', '01deg_jra55_SAMextr_1011_from9091']

In [ ]:
plt.figure(figsize=(10,8))

for expt in expts:
    mean_tau_x = calc_mean_tau_x(expt)
    plt.plot(mean_tau_x, mean_tau_x.yu_ocean,
             linewidth=2,
             label=expt)

plt.ylim([-75,-30])
plt.xlim([-0.02,0.2])
plt.ylabel('Latitude ($^\circ$N)')
plt.xlabel('Stress (N m$^{-2}$)')
plt.legend(loc=8, fontsize=10)
plt.title('Wind Stress Zonal Means')
plt.show()